In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Data Compression


Today's agenda:

- Huffman Coding

## Fixed-length encoding

Suppose we are given a document $D$ in which we use the alphabet $\Sigma = \{\sigma_1 \ldots \sigma_k\}$. Our goal is to create a binary encoding of $\Sigma$ to represent $D$ with as few bits as possible. Of course, the encoding must distinctly represent $\Sigma$.

> Example: Suppose $\Sigma=\{A, B, C, D\}$, and document $D = \langle A, A, A, A, A, A, A, A, A, B, C, D\rangle$. 

The naive encoding could be 

|$$\sigma$$|$e(\sigma)$         |
|-------|-----------------------|
| A     | 00 |
| B     | 01 |
| C     | 10 | 
| D     | 11 |

This is a **fixed-length** encoding of $\Sigma$. What is the number of bits required to encode the entire document with this encoding?


The length of the document with this encoding is $2\cdot 12 = 24$. The encoding is:

$e(D) = "000000000000000000011011"$

## Variable-length encoding


Fixed-length encoding doesn't account for redundancy in the document. 

Let $f: \sigma \rightarrow [0,1]$ be the frequency of the characters in $D$; this is easily computed in $O(|D|)$ work. What about the span?

Intuitively, we should encode the document by the frequency of the characters in the alphabet. The more frequent the character, the smaller its code should be.


$D = \langle A, A, A, A, A, A, A, A, A, B, C, D\rangle$

|$$\sigma$$ | $$f(\sigma)$$|
|-------|---------------|
| A     | 9 |
| B     | 1 |
| C     | 1 | 
| D     | 1 |



So, following that logic, we could come up with a code like this:

|$$\sigma$$ |$$e'(\sigma)$$|
|-------|---------------|
| A     | 0   |
| B     | 1  |
| C     | 00 | 
| D     | 11 |

Is this a valid encoding?

<br><br>
How should we decode `11`? It's ambiguous between `B` and `DD`.

<br><br>

Instead, we could use:


|$$\sigma$$ |$$e'(\sigma)$$|
|-------|---------------|
| A     | 0   |
| B     | 10  |
| C     | 110 | 
| D     | 111 |

This is a **variable-length** encoding, where each character may be encoded by a different number of bits.

This leads to an encoding of $D = \langle A, A, A, A, A, A, A, A, A, B, C, D\rangle$ as:

<br><br><br>

$e'(D) = "00000000010110111"$

<br><br>

This has length $1\cdot 9 + 2\cdot 1 + 3\cdot 1 + 3\cdot 1 = 17$. So this is a bit better. 

In general, the cost of a given encoding $e$ is 

$$C(e) = \sum_{i=0}^{|D|} |e(D[i])| = |D| \sum_{\sigma\in\Sigma} f(\sigma)\cdot e(\sigma).$$

Over all possible valid encodings $e: \Sigma \rightarrow \{0,1\}^*$, we want to find a variable-length encoding $e_*$ so that $C(e_*)$ is minimized.



## Encodings as Trees

How do we ensure that a variable-length encoding is valid? In other words, how do we only consider variable-length encodings that are *prefix-free*?

We can think of an encoding as representing a tree, with characters from $\Sigma$ as leaves. Note that a fixed-length encoding has all leaves at the same level. 

For the two encodings we gave, we'd have:

<img src = "encoding_trees.jpg" width="60%">

Every prefix-free encoding $e$ can be represented by a tree $T_e$. 

The depth of each character $d_T(\sigma)$ in the tree determines how many bits are needed to encode $\sigma$.

So the optimal compression of $D$ can be achieved by identifying the encoding tree $T$ that minimizes:

$$C(T) = |D|  \sum_{\sigma\in\Sigma} f(\sigma)\cdot d_T(\sigma)$$

We will come up with a greedy algorithm for constructing $T$ and show that it is optimal.


### Huffman Coding


Intuitively we know we should ensure that when constructing an encoding tree, the higher the frequency, the shorter the path length.

How about if we sort the frequencies in descending order and then assign tree positions in this order? But how do we guarantee the highest frequency characters have a short depth? 

We could group the characters into two sets of equal total frequency, this way the more frequent characters will have lower depth. This divide-and-conquer approach was developed by Shannon-Fano... but is not optimal.

Unfortunately Shannon-Fano coding is not provably optimal. David Huffman (as a graduate student in Robert Fano's class) came up with a *bottom-up* greedy algorithm as a class project and was able to prove that it was optimal.

The main idea of this algorithm is to choose the two **least** frequent characters $x$ and $y$ and create a subtree with $x$ and $y$ as sibling leaves for the final encoding. We then remove $x$ and $y$ from $\Sigma$ and add a *new* character $z$ with frequency $f(x)+f(y)$, and recurse to compute a tree $T'$. The final tree $T$ is just $T'$ with $z$ replaced by the subtree with $x, y$ as siblings. 

<img src="huffman_example.jpg" width="60%">


<br>

> How can we use a min-heap to implement this algorithm?

We need to efficiently retrieve the next two smallest frequency nodes.


![huffman-heap.png](huffman-heap.png)

1. Initialize a min-heap with character frequencies $f(\sigma)$

Then, repeat:

2. Call `deleteMin` twice to get the two least frequent nodes $x$ and $y$
3. Create a new node $z$ with frequency $f(x) + f(y)$
4. Make $x$ and $y$ children of $z$ in the tree.
4. Call `insert` to add $z$ to the heap

How many times will this repeat?

<br><br>

What is work/span of this algorithm?


Because we will always reduce the number of nodes by 1, this will repeat $n$ times (where $n = |\Sigma|$).

The cost of 2 calls to `deleteMin` and one call to `insert` is $3 \lg n$.

Thus, total work is $O(n \lg n)$.

We unfortunately have not exposed any parallelism in this algorithm, so the span is also $O(n \lg n)$.

### Proof of Optimality

**Greedy Choice**: Let $x, y\in\Sigma$ have the two smallest frequencies ($f(x) \leq f(y)$). Then there is an optimal encoding $T$ for $\Sigma$ with $x, y$ as sibling leaves at maximum depth. 

**Proof**: First let us observe that in an optimal solution $T^*$, $x$ must have maximum depth. Suppose that it did not, and some character $a$ with frequency $f(a)$ had depth $d_{T^*}(a) > d_{T^*}(x)$. Then we can construct a new tree $T^{**}$ with cost

$\begin{eqnarray*}
C(T^{**}) &=& C(T^*) - f(a)d_{T^*}(a)  - f(x)d_{T^*}(x) + f(x)d_{T^*}(a) + f(x)d_{T^*}(x) \\
&=& C(T^*) - (f(a)d_{T^*}(a) + f(x)d_{T^*}(x) - f(x)d_{T^*}(a) - f(x)d_{T^*}(x)) \\
&=& C(T^*) - (f(a)-f(x))(d_{T^*}(a) - d_{T^*}(x)) \\
\end{eqnarray*}$

If $f(a) > f(x)$ then $C(T^{**}) < C(T^*)$, which is a contradiction. So this means that we can swap $a$ and $x$ without changing the cost and thus some optimal solution has $x$ as a maximum depth leaf.

<img src = "huffman_exchange_argument.jpg" width = "30%">

Now why are $x$ and $y$ siblings? Well, first we can observe that every leaf in an optimal tree has a sibling. If not, we can reduce the encoding cost of some character. Now we can apply the same swapping argument as above to establish that $y$ is a sibling of $x$ in some optimal solution.


**Optimal Substructure**: Let $\Sigma' = \Sigma - \{x, y\} \cup \{z\}$, where $z\not\in \Sigma$ is a character with frequency $f(z) = f(x) + f(y)$. If $T'$ is an optimal encoding for $\Sigma'$, then an optimal encoding $T$ for $\Sigma$ can be constructed from $T'$ by replacing the leaf representing $z$ with an internal node that has $x, y$ as children. 

**Proof**: We first make a useful observation of the relationship between the cost of $T$ and $T'$. For simplicity, we'll drop $|D|$ from the definition of $C(T)$. Observe that:

$\begin{eqnarray*}
C(T) &=& \sum_{\sigma\in\Sigma} f(\sigma)d_T(\sigma) \\
&=& f(x)d_T(x) + f(y)d_T(y) + \sum_{\sigma\in\Sigma - \{x,y\}} f(\sigma)d_T(\sigma) \\
&=& (f(x)+f(y))(1+d_T(z)) + \sum_{\sigma\in\Sigma - \{x,y\}} f(\sigma)d_T(\sigma) \\
&=& (f(z))(1+d_T(z)) + \sum_{\sigma\in\Sigma - \{x,y\}} f(\sigma)d_T(\sigma) \\
&=& f(z) + C(T') \\
\end{eqnarray*}$

Suppose that $T$ was not optimal, and that there was some other tree $Z$ with $C(Z) < C(T)$. We can assume that $Z$ must have the two smallest frequency characters as siblings at maximum depth as established above. Now let's construct  
$Z'$ by removing $x, y$ and adding $z$ with $f(z) = f(x)+f(y)$). $Z'$ is an encoding for $\sigma'$, and we have that $C(Z) = f(z) + C(Z')$. By our assumption $C(Z) < C(T)$, so $f(z) + C(Z') < f(z) + C(T)$ using the observation above. But this is a contradiction to the optimality of $T'$ since we now have that $C(Z') < C(T').$

Thus Huffman coding produces an optimal prefix-free encoding. 


### decoding

So how do we use Huffman coding in practice?

Given a document $D$, we would compute frequencies and then construct an encoding tree $T$. Then to store the compressed version of $D$ we save the encoding of $\Sigma$, the tree $T$ and the encoded version of $D$. To decode a document, we would read bits sequentially and traverse $T$, emitting the appropriate character from $\Sigma$ whenever we hit a leaf. 

What is the worst-case work of decoding a document?

<img src = "encoding_trees.jpg" width="60%">

$00000000010110111 \rightarrow \langle A, A, A, A, A, A, A, A, A, B, C, D\rangle$

- We start at the root, and descend the tree until we reach a leaf.
- Then, we return to the root and repeat

So, if the input has $n$ bits, we will visit each bit exactly once.

$\Rightarrow O(n)$

which is equivalent to the total cost of the encoding tree:

$\Rightarrow O(C(T))$